In [2]:
!pip install -U awscli sagemaker boto3 --quiet

In [3]:
import boto3
import sagemaker
import sys

print(f"SageMaker Version: {sagemaker.__version__}")
print(f"Boto3 Version: {boto3.__version__}")

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
SageMaker Version: 2.208.0
Boto3 Version: 1.34.44


In [4]:
# Import required libraries
import os
import json
import itertools
import numpy as np
import pandas as pd

from datetime import datetime
from time import gmtime, strftime, sleep
from uuid import uuid4
from IPython import display

In [5]:
# Define region, bucket
session = sagemaker.Session()
region = boto3.Session().region_name
bucket = session.default_bucket()
# use the below for default SageMaker execution role else replace with your own IAM Role ARN
role = sagemaker.get_execution_role()

prefix = "autopilot/bankadditional"

today = datetime.now().strftime("%d%b%Y")
timestamp_suffix = f"{str(uuid4())[:6]}-{today}"

In [6]:
# Define sagemaker client object to invoke Sagemaker services
sm_client = boto3.client("sagemaker", region_name=region)

In [7]:
# Set prefix for AutoML jobnames.
automl_job_prefix = "bankmrkt" 
model_prefix = automl_job_prefix

print(f"Bucket: s3://{bucket}/{prefix}")
print(f"Region: {region}")
print(f"Role: {role}")
print(f"Job and model prefix string: {automl_job_prefix}")
print(f"suffix string: {timestamp_suffix}")

Bucket: s3://sagemaker-us-west-2-992382660999/autopilot/bankadditional
Region: us-west-2
Role: arn:aws:iam::992382660999:role/service-role/AmazonSageMaker-ExecutionRole-20240218T010479
Job and model prefix string: bankmrkt
suffix string: ffb634-17Feb2024


Download dataset from sagemaker-example-files-prod-{region} s3 bucket:

In [8]:
from sagemaker.s3 import S3Downloader

s3uri = f"s3://sagemaker-example-files-prod-{region}/datasets/tabular/uci_bank_marketing/bank-additional-full.csv"

if not os.path.exists('data/bank-additional/bank-additional-full.csv'):
    print("Downloading bank-additional-full.csv...")
    !mkdir -p data/bank-additional
    S3Downloader.download(s3_uri=s3uri, local_path="data/bank-additional", sagemaker_session=session)
    print("Done")
else:
    print("Skipping download..dataset exists at ./data/bank-additional")

Done


**Visualize dataset**

*The data is related with direct marketing campaigns (phone calls) of a Portuguese banking institution. The classification goal is to predict if the client will subscribe a term deposit (variable y).*

Problem Type: Binary Classification

In [9]:
df_data = pd.read_csv("./data/bank-additional/bank-additional-full.csv")

pd.set_option("display.max_columns", 10)  # View all of the columns
df_data  # show first 5 and last 5 rows of the dataframe

,age,job,marital,education,default,...,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,...,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,...,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,...,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,...,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,...,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,no,...,94.767,-50.8,1.028,4963.6,yes
41184,46,blue-collar,married,professional.course,no,...,94.767,-50.8,1.028,4963.6,no
41185,56,retired,married,university.degree,no,...,94.767,-50.8,1.028,4963.6,no
41186,44,technician,married,professional.course,no,...,94.767,-50.8,1.028,4963.6,yes


**Uploading dataset to S3**

In [10]:
# Set this flag to False for subsequent runs of this notebook
upload_dataset = True

In [11]:
DATA_FILE = "data/bank-additional/bank-additional-full.csv"

if upload_dataset:
    print(f"Uploading data to s3...")
    dataset_s3uri = session.upload_data(DATA_FILE, key_prefix=f"{prefix}/raw")
    print(f"Data uploaded to : \n {dataset_s3uri}")
else:
    dataset_s3uri = f"s3://{bucket}/{prefix}/raw/bank-additional-full.csv"
    print(f"Skipping upload .. dataset is under: {dataset_s3uri}")

Uploading data to s3...
Data uploaded to : 
 s3://sagemaker-us-west-2-992382660999/autopilot/bankadditional/raw/bank-additional-full.csv


**Launching Autopilot jobs in ENSEMBLING and HPO modes**

In [13]:
# Autopilot job params
target_column = "y"
training_mode = "ENSEMBLING"

# Optional Parameters
problem_type = "BinaryClassification"
objective_metric = "F1"
max_job_runtime_seconds = 3600
max_runtime_per_job_seconds = 1200
max_candidates = 10

**Defining Autopilot job config values**

In [14]:
automl_job_config = {
    "CompletionCriteria": {
        "MaxRuntimePerTrainingJobInSeconds": max_runtime_per_job_seconds,
        "MaxAutoMLJobRuntimeInSeconds": max_job_runtime_seconds,
    },
    "Mode": training_mode,
}

automl_job_objective = {"MetricName": objective_metric}

input_data_config = [
    {
        "DataSource": {"S3DataSource": {"S3DataType": "S3Prefix", "S3Uri": dataset_s3uri}},
        "TargetAttributeName": target_column,
    }
]

output_data_config = {"S3OutputPath": f"s3://{bucket}/{prefix}/output"}

# Optional: Define a Tag
tags_config = [{"Key": "Project", "Value": "Autopilot-serverless"}]

**Launch Autopilot job with training mode set to ENSEMBLING**

In [16]:
try:
    ens_automl_job_name = f"{model_prefix}-ENS-{timestamp_suffix}"
    print(f"Launching AutoMLJob → {ens_automl_job_name} with mode set to {training_mode}")
    response = sm_client.create_auto_ml_job(
        AutoMLJobName=ens_automl_job_name,
        InputDataConfig=input_data_config,
        OutputDataConfig=output_data_config,
        AutoMLJobConfig=automl_job_config,
        ProblemType=problem_type,
        AutoMLJobObjective=automl_job_objective,
        RoleArn=role,
        Tags=tags_config,
    )
    print(response)
except Exception as e:
    print(f"Error launching ENSEMBLING Autopilot Job: {ens_automl_job_name}")
    print(f"{e}")
    pass

Launching AutoMLJob → bankmrkt-ENS-ffb634-17Feb2024 with mode set to ENSEMBLING
{'AutoMLJobArn': 'arn:aws:sagemaker:us-west-2:992382660999:automl-job/bankmrkt-ENS-ffb634-17Feb2024', 'ResponseMetadata': {'RequestId': '5844c8ab-012c-4f40-aa8b-382bb3e5f83d', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '5844c8ab-012c-4f40-aa8b-382bb3e5f83d', 'content-type': 'application/x-amz-json-1.1', 'content-length': '100', 'date': 'Sat, 17 Feb 2024 20:07:03 GMT'}, 'RetryAttempts': 0}}


**Launching Autopilot job with training mode set to HYPERPARAMETER_TUNING mode**

In [17]:
# We use the defined job prefix to construct model name(s) and later to construct endpoint config and endpoint names.
try:
    training_mode = "HYPERPARAMETER_TUNING"
    automl_job_config["Mode"] = training_mode
    automl_job_config["CompletionCriteria"]["MaxCandidates"] = 15
    hpo_automl_job_name = f"{model_prefix}-HPO-{timestamp_suffix}"
    print(f"Launching AutoMLJob → {hpo_automl_job_name} with mode set to {training_mode}")
    response = sm_client.create_auto_ml_job(
        AutoMLJobName=hpo_automl_job_name,
        InputDataConfig=input_data_config,
        OutputDataConfig=output_data_config,
        AutoMLJobConfig=automl_job_config,
        ProblemType=problem_type,
        AutoMLJobObjective=automl_job_objective,
        RoleArn=role,
        Tags=tags_config,
    )
    print(response)
except Exception as e:
    print(f"Error launching HPO Autopilot Job: {hpo_automl_job_name}")
    print(f"{e}")
    pass

Launching AutoMLJob → bankmrkt-HPO-ffb634-17Feb2024 with mode set to HYPERPARAMETER_TUNING
{'AutoMLJobArn': 'arn:aws:sagemaker:us-west-2:992382660999:automl-job/bankmrkt-HPO-ffb634-17Feb2024', 'ResponseMetadata': {'RequestId': '99b186b3-29a5-43a8-9b65-28095733e933', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '99b186b3-29a5-43a8-9b65-28095733e933', 'content-type': 'application/x-amz-json-1.1', 'content-length': '100', 'date': 'Sat, 17 Feb 2024 20:07:49 GMT'}, 'RetryAttempts': 0}}


**Monitoring AutoML job completion status**

In [18]:
def get_job_status(sm_client, job_name):
    resp = sm_client.describe_auto_ml_job(AutoMLJobName=job_name)
    p_status = resp["AutoMLJobStatus"]
    s_status = resp["AutoMLJobSecondaryStatus"]
    desc = f"{job_name}: {p_status} | {s_status} ..."
    return (p_status, desc)

In [19]:
# monitor job status launched in ensembling mode
(p_status, desc) = get_job_status(sm_client, ens_automl_job_name)

while p_status not in ("Completed", "Failed"):
    (p_status, desc) = get_job_status(sm_client, ens_automl_job_name)
    if p_status not in ("Completed", "Failed"):
        print(desc)
        sleep(60)
        continue
    else:
        break

bankmrkt-ENS-ffb634-17Feb2024: InProgress | TrainingModels ...
bankmrkt-ENS-ffb634-17Feb2024: InProgress | TrainingModels ...
bankmrkt-ENS-ffb634-17Feb2024: InProgress | TrainingModels ...
bankmrkt-ENS-ffb634-17Feb2024: InProgress | TrainingModels ...
bankmrkt-ENS-ffb634-17Feb2024: InProgress | TrainingModels ...
bankmrkt-ENS-ffb634-17Feb2024: InProgress | TrainingModels ...
bankmrkt-ENS-ffb634-17Feb2024: InProgress | TrainingModels ...
bankmrkt-ENS-ffb634-17Feb2024: InProgress | TrainingModels ...
bankmrkt-ENS-ffb634-17Feb2024: InProgress | TrainingModels ...
bankmrkt-ENS-ffb634-17Feb2024: InProgress | TrainingModels ...
bankmrkt-ENS-ffb634-17Feb2024: InProgress | TrainingModels ...
bankmrkt-ENS-ffb634-17Feb2024: InProgress | TrainingModels ...
bankmrkt-ENS-ffb634-17Feb2024: InProgress | TrainingModels ...
bankmrkt-ENS-ffb634-17Feb2024: InProgress | TrainingModels ...
bankmrkt-ENS-ffb634-17Feb2024: InProgress | TrainingModels ...
bankmrkt-ENS-ffb634-17Feb2024: InProgress | TrainingMod

In [94]:
def create_autopilot_model(sm_client, model_name, role, model_container, index):
    try:
        transform_mode = model_container["Environment"]["AUTOML_TRANSFORM_MODE"]
        if transform_mode:
            model_name = f"{model_name}-datamodel-{index}"
    except:
        model_name = f"{model_name}-Inf-{index}"

    if len(model_name) <= 63:
        print(f"Creating Model {index}: {model_name} ...")
        model_response = sm_client.create_model(
            ModelName=model_name, ExecutionRoleArn=role, Containers=[model_container]
        )
        status_code = model_response["ResponseMetadata"]["HTTPStatusCode"]
        model_arn = model_response["ModelArn"]
        return (status_code, model_arn)
    else:
        print(f"Model Name: {model_name} length exceeds max. allowed chars : 63")
        raise ValueError("Model name cannot exceed 63 chars.")


def create_serverless_endpoint_config(
    sm_client, endpoint_config_name, model_name, memory: int = 2048, max_concurrency: int = 20
):
    if len(endpoint_config_name) <= 63:
        print(f"Creating Endpoint Config: {endpoint_config_name} ...")
        try:
            epc_response = sm_client.create_endpoint_config(
                EndpointConfigName=endpoint_config_name,
                ProductionVariants=[
                    {
                        "ModelName": model_name,
                        "VariantName": "AllTraffic",
                        "ServerlessConfig": {
                            "MemorySizeInMB": memory,
                            "MaxConcurrency": max_concurrency,
                        },
                    }
                ],
            )
            status_code = epc_response["ResponseMetadata"]["HTTPStatusCode"]
            epc_arn = epc_response["EndpointConfigArn"]
            return (status_code, epc_arn)
        except Exception as e:
            print(f"Error creating EndpointConfig: {endpoint_config_name}")
            print(f"{e}")
    else:
        print(f"EndpointConfig name exceeds allowed 63 char limit")
        raise ValueError("EndpointConfig name cannot exceed 63 chars.")


def create_serverless_endpoint(sm_client, endpoint_name, endpoint_config_name):
    if len(endpoint_name) <= 63:
        print(f"Creating Serverless Endpoint: {endpoint_name} ...")
        try:
            ep_response = sm_client.create_endpoint(
                EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
            )
            status_code = ep_response["ResponseMetadata"]["HTTPStatusCode"]
            return status_code
        except Exception as e:
            print(f"Error creating Endpoint: {endpoint_name}")
            print(f"{e}")
    else:
        print(f"Endpoint name exceeds allowed 63 char limit")
        raise ValueError("Endpoint name cannot exceed 63 chars.")


def get_s3_objsize_in_MB(bucket, key):
    s3 = boto3.client("s3")
    resp = s3.head_object(Bucket=bucket, Key=key)["ContentLength"]
    size = round(resp / (1024 * 1024))
    if size < 1:
        print(f"Model Size: ~ {round(resp / 1024)} KB")
    else:
        print(f"Model Size: ~ {size} MB")

    return size


def set_serverless_endpoint_memory(model_size: int):
    if model_size <= 1024:
        return 1024
    elif model_size > 1024 and model_size <= 2048:
        return 2048
    elif model_size > 2048 and model_size <= 3072:
        return 3072
    elif model_size > 3072 and model_size <= 4096:
        return 4096
    elif model_size > 4096 and model_size <= 5120:
        return 5120
    elif model_size > 5120 and model_size <= 6144:
        return 6144
    elif model_size > 6144:
        raise ValueError("Model size is greater than 6GB")

In [95]:
response = sm_client.describe_auto_ml_job(AutoMLJobName=ens_automl_job_name)
inference_container = response["BestCandidate"]["InferenceContainers"][0]
print(f"Inference Container for AutoML job: {ens_automl_job_name}")
print(inference_container)

# Verify generated model size before creating endpoint config.
# Extract s3 Key from ModelDataUrl
model_dataurl_key = inference_container["ModelDataUrl"].split(f"{bucket}")[1][1:]
ens_model_size = get_s3_objsize_in_MB(bucket, model_dataurl_key)
print(f"Ensemble Model Size: ~ {ens_model_size}MB")

Inference Container for AutoML job: bankmrkt-ENS-ffb634-17Feb2024
{'Image': '763104351884.dkr.ecr.us-west-2.amazonaws.com/autogluon-inference:0.4.3-cpu-py38-ubuntu20.04', 'ModelDataUrl': 's3://sagemaker-us-west-2-992382660999/autopilot/bankadditional/output/bankmrkt-ENS-ffb634-17Feb2024/sagemaker-automl-candidates/model/WeightedEnsemble-L2-FULL-t3/model.tar.gz', 'Environment': {'MODEL_NAME': 'WeightedEnsemble-L2-FULL', 'SAGEMAKER_DEFAULT_INVOCATIONS_ACCEPT': 'text/csv', 'SAGEMAKER_INFERENCE_OUTPUT': 'predicted_label', 'SAGEMAKER_INFERENCE_SUPPORTED': 'predicted_label,probability,probabilities,labels', 'SAGEMAKER_PROGRAM': 'tabular_serve.py', 'SAGEMAKER_SUBMIT_DIRECTORY': '/opt/ml/model/code'}}
Model Size: ~ 3 MB
Ensemble Model Size: ~ 3MB


In [96]:
models = list()
# create model
(status, model_arn) = create_autopilot_model(
    sm_client, ens_automl_job_name, role, inference_container, 0
)
model_name = model_arn.split("/")[1]
models.append(model_name)

endpoint_configs = list()
endpoint_config_name = f"epc-{model_name}"
memory = 3072
# create endpoint config
(status, epc_arn) = create_serverless_endpoint_config(
    sm_client, endpoint_config_name, model_name, memory=memory, max_concurrency=10
)
endpoint_configs.append(endpoint_config_name)

endpoints = list()
endpoint_name = endpoint_config_name.replace("epc-", "ep-")
# create serverless endpoint
create_serverless_endpoint(sm_client, endpoint_name, endpoint_config_name)
endpoints.append(endpoint_name)

Creating Model 0: bankmrkt-ENS-ffb634-17Feb2024-Inf-0 ...
Creating Endpoint Config: epc-bankmrkt-ens-ffb634-17feb2024-inf-0 ...
Creating Serverless Endpoint: ep-bankmrkt-ens-ffb634-17feb2024-inf-0 ...


In [97]:
def get_endpoint_status(sm_client, endpoint_name):
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    desc = f"{endpoint_name} | {status} ..."
    return (status, desc)

In [98]:
# monitor endpoint status
(status, desc) = get_endpoint_status(sm_client, endpoint_name)
print(desc)
while status not in ("InService", "Failed"):
    (status, desc) = get_endpoint_status(sm_client, endpoint_name)
    if status not in ("InService", "Failed"):
        print(desc)
        sleep(60)
        continue
    else:
        print(desc)
        break

ep-bankmrkt-ens-ffb634-17feb2024-inf-0 | InService ...


**Sending Inference request to serverless endpoint with ENSEMBLE model**

In [99]:
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import CSVDeserializer

endpoint = endpoints[0]

payload = "51,technician,married,professional.course,no,yes,no,cellular,apr,thu,687,1,0,1,success,-1.8,93.075,-47.1,1.365,5099.1"
# payload = "42,services,married,professional.course,no,yes,no,telephone,may,thu,813,1,999,0,nonexistent,1.1,93.994,-36.4,4.855,5191.0"
# payload = "37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0"
# payload = "55,admin.,married,high.school,no,no,no,telephone,may,thu,94,1,999,0,nonexistent,1.1,93.994,-36.4,4.855,5191.0"
# payload = "34,blue-collar,married,basic.4y,no,no,no,telephone,may,tue,800,4,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0"

try:
    print(f"Invoking endpoint: {endpoint} with payload .. \n")
    print(payload)
    predictor = Predictor(
        endpoint_name=endpoint,
        sagmaker_session=session,
        serializer=CSVSerializer(),
        deserializer=CSVDeserializer(),
    )
    prediction = predictor.predict(payload)
    print(f"Predicted Label: {prediction[0][0]}")
except Exception as e:
    print(f"Error invoking Endpoint: {endpoint}")
    print(f"{e}")
    pass

Invoking endpoint: ep-bankmrkt-ens-ffb634-17feb2024-inf-0 with payload .. 

51,technician,married,professional.course,no,yes,no,cellular,apr,thu,687,1,0,1,success,-1.8,93.075,-47.1,1.365,5099.1
Predicted Label: yes


**Cleanup (ensemble endpoint)**

Deleting endpoint, endpoint config and model in that order


In [100]:
epc_name = endpoint.replace("ep-", "epc-")
model_name = endpoint.replace("ep-", "")

print(f"Deleting endpoint : {endpoint}")
try:
    sm_client.delete_endpoint(EndpointName=endpoint)
except Exception as e:
    print(f"{e}")
    pass

print(f"Deleting EndpointConfig : {epc_name}")
try:
    sm_client.delete_endpoint_config(EndpointConfigName=epc_name)
except Exception as e:
    print(f"{e}")
    pass

print(f"Deleting Model : {model_name}")
try:
    sm_client.delete_model(ModelName=model_name)
except Exception as e:
    print(f"{e}")
    pass

Deleting endpoint : ep-bankmrkt-ens-ffb634-17feb2024-inf-0
Deleting EndpointConfig : epc-bankmrkt-ens-ffb634-17feb2024-inf-0
Deleting Model : bankmrkt-ens-ffb634-17feb2024-inf-0


***Deploying HPO models to serverless endpoints***

Autopilot in HYPERPARAMETER_TUNING mode generates 3 inference containers for binary classification problem types.


In [101]:
# monitor job status launched in hpo mode
(p_status, desc) = get_job_status(sm_client, hpo_automl_job_name)
print(desc)
while p_status not in ("Completed", "Failed"):
    (p_status, desc) = get_job_status(sm_client, hpo_automl_job_name)
    if p_status not in ("Completed", "Failed"):
        print(desc)
        sleep(60)
        continue
    else:
        print(desc)
        break

bankmrkt-HPO-ffb634-17Feb2024: Completed | Completed ...


In [102]:
job_response = sm_client.describe_auto_ml_job(AutoMLJobName=hpo_automl_job_name)
inference_containers = job_response["BestCandidate"]["InferenceContainers"]
print(f"Inference Containers for AutoML job: {hpo_automl_job_name}")
print(inference_containers)

Inference Containers for AutoML job: bankmrkt-HPO-ffb634-17Feb2024
[{'Image': '246618743249.dkr.ecr.us-west-2.amazonaws.com/sagemaker-sklearn-automl:2.5-1-cpu-py3', 'ModelDataUrl': 's3://sagemaker-us-west-2-992382660999/autopilot/bankadditional/output/bankmrkt-HPO-ffb634-17Feb2024/data-processor-models/bankmrkt-HPO-ffb634-17Feb2024-dpp4-1-54efc9be7a8f479cb1c3c6626f/output/model.tar.gz', 'Environment': {'AUTOML_SPARSE_ENCODE_RECORDIO_PROTOBUF': '1', 'AUTOML_TRANSFORM_MODE': 'feature-transform', 'SAGEMAKER_DEFAULT_INVOCATIONS_ACCEPT': 'application/x-recordio-protobuf', 'SAGEMAKER_PROGRAM': 'sagemaker_serve', 'SAGEMAKER_SUBMIT_DIRECTORY': '/opt/ml/model/code'}}, {'Image': '246618743249.dkr.ecr.us-west-2.amazonaws.com/sagemaker-xgboost:1.3-1-cpu-py3', 'ModelDataUrl': 's3://sagemaker-us-west-2-992382660999/autopilot/bankadditional/output/bankmrkt-HPO-ffb634-17Feb2024/tuning/bankmrkt-H-dpp4-xgb/bankmrkt-HPO-ffb634-17Feb2024nuh-012-c2a73931/output/model.tar.gz', 'Environment': {'MAX_CONTENT_L

In [103]:
for idx, container in enumerate(inference_containers):
    print(f"calculating generated model_{idx} size")
    # Extract s3 Key from ModelDataUrl
    model_dataurl_key = container["ModelDataUrl"].split(f"{bucket}")[1][1:]
    # print(model_dataurl_key)
    model_size = get_s3_objsize_in_MB(bucket, model_dataurl_key)

calculating generated model_0 size
Model Size: ~ 13 KB
calculating generated model_1 size
Model Size: ~ 1 MB
calculating generated model_2 size
Model Size: ~ 13 KB


In [104]:
models = list()
endpoint_configs = list()
endpoints = list()

memory = 2048
max_concurreny = 10

# Create model, endpoint_config, endpoint and store them in lists for easier access
for idx, container in enumerate(inference_containers):
    (status, model_arn) = create_autopilot_model(
        sm_client, hpo_automl_job_name, role, container, idx
    )
    model_name = model_arn.split("/")[1]
    print(f"\tcreated model: {model_name}...")
    models.append(model_name)

    endpoint_config_name = f"epc-{model_name}"
    endpoint_name = f"ep-{model_name}"

    (status, epc_arn) = create_serverless_endpoint_config(
        sm_client, endpoint_config_name, model_name, memory=memory, max_concurrency=max_concurreny
    )
    print(f"\tcreated epc: {endpoint_config_name}")
    endpoint_configs.append(endpoint_config_name)

    res = create_serverless_endpoint(sm_client, endpoint_name, endpoint_config_name)
    print(f"\tcreated ep: {endpoint_name}")
    endpoints.append(endpoint_name)

Creating Model 0: bankmrkt-HPO-ffb634-17Feb2024-datamodel-0 ...
	created model: bankmrkt-hpo-ffb634-17feb2024-datamodel-0...
Creating Endpoint Config: epc-bankmrkt-hpo-ffb634-17feb2024-datamodel-0 ...
	created epc: epc-bankmrkt-hpo-ffb634-17feb2024-datamodel-0
Creating Serverless Endpoint: ep-bankmrkt-hpo-ffb634-17feb2024-datamodel-0 ...
	created ep: ep-bankmrkt-hpo-ffb634-17feb2024-datamodel-0
Creating Model 1: bankmrkt-HPO-ffb634-17Feb2024-Inf-1 ...
	created model: bankmrkt-hpo-ffb634-17feb2024-inf-1...
Creating Endpoint Config: epc-bankmrkt-hpo-ffb634-17feb2024-inf-1 ...
	created epc: epc-bankmrkt-hpo-ffb634-17feb2024-inf-1
Creating Serverless Endpoint: ep-bankmrkt-hpo-ffb634-17feb2024-inf-1 ...
Error creating Endpoint: ep-bankmrkt-hpo-ffb634-17feb2024-inf-1
An error occurred (ResourceLimitExceeded) when calling the CreateEndpoint operation: The account-level service limit 'Maximum total concurrency that can be allocated across all serverless endpoints' is 10 Number, with current ut

In [111]:
import boto3
from time import sleep

# Initialize the boto3 SageMaker client
sm_client = boto3.client('sagemaker', region_name='us-west-2')

# Replace with your actual endpoint names
endpoints = ['ep-bankmrkt-hpo-ffb634-17feb2024-datamodel-0']

# Function to get the status of the endpoint
def get_endpoint_status(client, endpoint_name):
    response = client.describe_endpoint(EndpointName=endpoint_name)
    return response['EndpointStatus']

# Get the initial statuses for all endpoints
statuses = [get_endpoint_status(sm_client, ep) for ep in endpoints]
print(statuses)

# Continue looping until the endpoint is in service
while statuses != ["InService"]:  # Adjust the condition based on the actual number of endpoints
    statuses = [get_endpoint_status(sm_client, ep) for ep in endpoints]
    print(statuses)
    
    # If the status is not "InService", wait for 60 seconds before checking again
    if statuses != ["InService"]:
        sleep(60)
    else:
        print("All endpoints are in service.")
        break


['InService']


In [116]:
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer, IdentitySerializer
from sagemaker.deserializers import CSVDeserializer

# Assuming 'endpoints' is a list of your endpoint names and 'session' is a valid SageMaker session
payload = "51,technician,married,professional.course,no,yes,no,cellular,apr,thu,687,1,0,1,success,-1.8,93.075,-47.1,1.365,5099.1"

for idx, ep in enumerate(endpoints):
    try:
        print(f"Payload: {payload}")
        # Check your logic for when to use IdentitySerializer vs. CSVSerializer
        if idx == 1:
            predictor = Predictor(
                endpoint_name=ep,
                sagemaker_session=session,
                serializer=IdentitySerializer(content_type="application/x-recordio-protobuf"),
                deserializer=CSVDeserializer(),
            )
        else:
            predictor = Predictor(
                endpoint_name=ep, sagemaker_session=session, serializer=CSVSerializer(),
                deserializer=CSVDeserializer()  # Assuming the endpoint returns CSV formatted string
            )
        prediction = predictor.predict(payload)
        print(f"Prediction for endpoint {ep}: \n{prediction}")
        print("--" * 20)
    except Exception as e:
        print(f"Error invoking Endpoint: {ep} \n {e}")



Payload: 51,technician,married,professional.course,no,yes,no,cellular,apr,thu,687,1,0,1,success,-1.8,93.075,-47.1,1.365,5099.1
Error invoking Endpoint: ep-bankmrkt-hpo-ffb634-17feb2024-datamodel-0 
 An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (500) from model with message "<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>500 Internal Server Error</title>
<h1>Internal Server Error</h1>
<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>
". See https://us-west-2.console.aws.amazon.com/cloudwatch/home?region=us-west-2#logEventViewer:group=/aws/sagemaker/Endpoints/ep-bankmrkt-hpo-ffb634-17feb2024-datamodel-0 in account 992382660999 for more information.


**Cleanup (HPO endpoints)**

In [123]:
print("Deleting endpoints...")
for _, ep in enumerate(endpoints):
    try:
        print(f"\tDeleting {ep}...")
        sm_client.delete_endpoint(EndpointName=ep)
    except Exception as e:
        print(f"{e}")
        continue
print("--" * 15)
print("Deleting endpoint configs...")
for _, epc in enumerate(endpoint_configs):
    try:
        print(f"\tDeleting {epc} ...")
        sm_client.delete_endpoint_config(EndpointConfigName=epc)
    except Exception as e:
        print(f"{e}")
        continue
print("--" * 15)
print("Deleting models...")
for _, mdl in enumerate(models):
    try:
        print(f"\tDeleting {mdl}...")
        sm_client.delete_model(ModelName=mdl)
    except Exception as e:
        print(f"{e}")
        continue

print(f"Done")

Deleting endpoints...
	Deleting ep-bankmrkt-hpo-ffb634-17feb2024-datamodel-0...
An error occurred (ValidationException) when calling the DeleteEndpoint operation: Could not find endpoint "ep-bankmrkt-hpo-ffb634-17feb2024-datamodel-0".
------------------------------
Deleting endpoint configs...
	Deleting epc-bankmrkt-hpo-ffb634-17feb2024-datamodel-0 ...
An error occurred (ValidationException) when calling the DeleteEndpointConfig operation: Could not find endpoint configuration "epc-bankmrkt-hpo-ffb634-17feb2024-datamodel-0".
	Deleting epc-bankmrkt-hpo-ffb634-17feb2024-inf-1 ...
An error occurred (ValidationException) when calling the DeleteEndpointConfig operation: Could not find endpoint configuration "epc-bankmrkt-hpo-ffb634-17feb2024-inf-1".
	Deleting epc-bankmrkt-hpo-ffb634-17feb2024-datamodel-2 ...
An error occurred (ValidationException) when calling the DeleteEndpointConfig operation: Could not find endpoint configuration "epc-bankmrkt-hpo-ffb634-17feb2024-datamodel-2".
---------